In [ ]:
# !pip install datasets
# from datasets import load_dataset

# dataset = load_dataset("cfilt/iitb-english-hindi")


# with open('eng-hin-train.txt' , 'w+' , encoding = "utf8") as file:
#     file.write("ENGLISH_SENTENCE_sep_HINDI_SENTENCE")
#     file.write("\n")
#     for translation_pair in dataset["train"]["translation"]:
#         source_sentence = translation_pair["en"]
#         target_sentence = translation_pair["hi"]
#         file.write(source_sentence.strip() + "_sep_")
#         file.write(target_sentence.strip() + "\n")

In [ ]:
# !wget https://storage.googleapis.com/drive-bulk-export-anonymous/20231127T081635.493Z/4133399871716478688/f4c6d602-f7e9-4be8-b29c-0ab031d52074/1/2d45123d-fab0-403e-b62d-be0ca5b37baa?authuser

--2023-11-27 08:18:36--  https://storage.googleapis.com/drive-bulk-export-anonymous/20231127T081635.493Z/4133399871716478688/f4c6d602-f7e9-4be8-b29c-0ab031d52074/1/2d45123d-fab0-403e-b62d-be0ca5b37baa?authuser
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 74.125.137.207, 142.250.101.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18045628 (17M) [application/x-zip]
Saving to: ‘2d45123d-fab0-403e-b62d-be0ca5b37baa?authuser’

2d45123d-fab0-403e- 100%[===================>]  17.21M  46.3MB/s    in 0.4s    

2023-11-27 08:18:37 (46.3 MB/s) - ‘2d45123d-fab0-403e-b62d-be0ca5b37baa?authuser’ saved [18045628/18045628]



In [ ]:
# !unzip test.zip

Archive:  test.zip
  inflating: eng-hin-train-200000.txt  


**Data loading and Preprocessing**


*   We will load all the files directly from the Google drive. So first we have to mount the drive into collab notebook.

*   For preprocessing we will only use csv module of python.


In [ ]:
import csv
import string
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#setting the device to "cuda" if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**Data Preprocessing**

For data preprocessing, we will do following steps:

1. Do not process sentences whose length is less than the MAX_LENGTH
2. Do not process any null pair
3. Converting into lowercase characters
4. Removing all punctutations from the sentences.
5. Removing the sentences who have some "english" words in hindi sentences.
6. Change the encoding of the sentences.
7. Build vocabulary for hindi and english languages to map index to unique words and vice versa.
8. Converting sentence into tensors for further processing

In [ ]:
MAX_LENGTH=20

class Vocab_builder:
    def __init__(self):
        self.word_2_index={"<SOS>":0,"<EOS>":1,"<PAD>":2,"<UKN>":3}
        self.index_2_word={0:"<SOS>", 1:"<EOS>", 2:"<PAD>", 3:"<UKN>"}
        self.freq={}
        self.size=4

    def add_this_sentence(self,sentence):
        words=sentence.split(" ")
        for word in words:
            if word not in self.word_2_index:
                #If the word is not there, add it to a new index and store the indexes
                #Initialize the frequency of the word to 1 and increase the size of the vocabulary
                self.word_2_index[word]=self.size
                self.freq[word]=1
                self.index_2_word[self.size]=word
                self.size+=1
            else:
                # If the word is already present then just increase the frequency
                self.freq[word]+=1

In [ ]:
#Initilizing the objects of hindi and english vocabularies:
hindi_vocab=Vocab_builder()
eng_vocab=Vocab_builder()

In [ ]:
def length(sentence):
    '''
        Function to tell the length of a sentence.
    '''
    return len(sentence.split(" "))

def is_mixed(sentence):
    '''
        This function will return True if a hindi sentence is containing some english character.
    '''
    letters="abcdefghijklmnopqrstuvwxyz"
    for ch in letters:
        if ch in sentence:
            return True
    return False

def preprocess(sentence):
    '''
        This function will apply the neccesary preprocessing to a sentence
    '''
    #First we will remove all punctuations from the sentence
    punctuations=list(string.punctuation)
    cleaned=""
    for letter in sentence:
        if letter not in punctuations:
            cleaned+=letter
    cleaned=cleaned.lower() ## Converting into lowercase
    return cleaned

In [ ]:
def clean_the_data(path):
    '''
      This function will load the data and process it line by line.
      It will apply all the preprocessing and make the data ready for further processing.
    '''
    pairs=[]
    with open(path,'rt') as f:
        data=f.readlines()
        row_num=0
        for row in data:
            if row_num!=0:  #We will not process first row as it will contain header
                unfiltered_sentences = row.split("_sep_")
                hindi=unfiltered_sentences[1]
                eng=unfiltered_sentences[0]

                if length(hindi)>=MAX_LENGTH or length(eng)>=MAX_LENGTH:  #skipping if length is more than MAX_LENGTH
                    continue
                if not hindi or not eng:  #skipping pair having any NULL value
                    continue
                if is_mixed(hindi):   #skipping sentence if it contains some english word
                    continue
                hindi=hindi.encode('utf-8',errors='ignore').decode('utf-8')
                eng=eng.encode('ascii',errors='ignore').decode('utf-8')
                hindi=preprocess(hindi)
                eng=preprocess(eng)
                #Adding <SOS>, <EOS> and padding tokens
                pair=[hindi.strip(), eng.strip()]

                hin_extra=MAX_LENGTH-len(hindi.strip().split(" "))
                eng_extra=MAX_LENGTH-len(eng.strip().split(" "))

                hindi_vocab.add_this_sentence(pair[0])
                eng_vocab.add_this_sentence(pair[1])
                pair[0]=pair[0].split(" ")
                pair[0].insert(0,"<SOS>")
                pair[0].append("<EOS>")
                pair[0]=pair[0]+["<PAD>"]*(hin_extra)

                pair[1]=pair[1].split(" ")
                pair[1].insert(0,"<SOS>")
                pair[1].append("<EOS>")
                pair[1]=pair[1]+["<PAD>"]*(eng_extra)

                pair[0]=" ".join(pair[0])
                pair[1]=" ".join(pair[1])
                pairs.append(pair)
            row_num+=1
    return pairs

In [ ]:
train_file_path="eng-hin-train-200000.txt"

In [ ]:
pairs=clean_the_data(train_file_path)

In [ ]:
print(pairs[11])
print(len(pairs))

['<SOS> छींटा <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>', '<SOS> pitterpatter <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>']
135864


In [ ]:
#Now we need to convert each of this pair into corresponding tensors
def pair_to_tensor(pair):
    '''
    A function to convert a given pair to tensors corresponding to index in vocabulary
    '''
    hindi_sentence=pair[0]
    eng_sentence=pair[1]
    indexes_hindi=[hindi_vocab.word_2_index[word] for word in hindi_sentence.split(' ')]
    indexes_eng=[eng_vocab.word_2_index[word] for word in eng_sentence.split(' ')]
    hindi_tensor=torch.tensor(indexes_hindi, dtype=torch.long, device=device).view(-1,1)
    eng_tensor=torch.tensor(indexes_eng, dtype=torch.long, device=device).view(-1,1)
    return (hindi_tensor, eng_tensor)

In [ ]:
hin_tensors=[]
eng_tensors=[]
for pair in pairs:      # we will convert each pair into tensor to process it
    hin,eng=pair_to_tensor(pair)
    hin_tensors.append(hin)
    eng_tensors.append(eng)

**Transformers**

References:
1. https://arxiv.org/abs/1706.03762
2. https://www.youtube.com/watch?v=iDulhoQ2pro
3. https://www.youtube.com/watch?v=TQQlZhbC5ps&t=636s
4. https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html

In [ ]:
class Transformer_model(nn.Module):
    def __init__(self, embed_size, len_src_vocab, len_tgt_vocab, src_pad_index, num_heads, enc_layers, dec_layers, forward_exp, dropout, max_length,device):
        super(Transformer_model,self).__init__()
        self.src_word_embedding=nn.Embedding(len_src_vocab, embed_size) #shape: (len_src_vocab, embed_size)
        self.tgt_word_embedding=nn.Embedding(len_tgt_vocab, embed_size) #shape: (len_eng_vocab, embed_size)
        self.src_positional_embedding=nn.Embedding(max_length, embed_size) #shape: (MAX_LENGTH, embed_size)
        self.tgt_positional_embedding=nn.Embedding(max_length, embed_size)  #shape: (MAX_LENGTH, embed_size)
        self.device=device
        self.transformer_layer=nn.Transformer(embed_size, num_heads,enc_layers, dec_layers, forward_expansion, dropout)
        self.out_fc=nn.Linear(embed_size, len_tgt_vocab)    #linear layer to predicted the output word
        self.dropout=nn.Dropout(dropout)
        self.src_pad_index=src_pad_index

    def gen_mask_for_src(self, source):
        #need to transpose source as padding need to be of size (batch_size, seq_len) but source is of shape (seq_len, batch_size)
        source=source.transpose(0,1)
        mask=(source==self.src_pad_index) #(mask will contain 1 where there is pad token, and 0 otherwise)
        return mask.to(self.device)

    def forward(self, src, target):
        src_seq_length, batch_size=src.shape
        tgt_seq_length, batch_size=target.shape
        # creating positional embeddings to encode position of words in transformer (it will be just a range array upto max_length)
        src_positional=torch.arange(0,src_seq_length).unsqueeze(1).expand(src_seq_length, batch_size).to(self.device)
        tgt_positional=torch.arange(0,tgt_seq_length).unsqueeze(1).expand(tgt_seq_length, batch_size).to(self.device)
        # calculating embeddings as sum of positional and word embeddings
        src_embedding=self.dropout(self.src_word_embedding(src)+self.src_positional_embedding(src_positional))
        tht_embedding=self.dropout(self.tgt_word_embedding(target)+self.tgt_positional_embedding(tgt_positional))
        # generating padding mask for hindi (source)
        src_padding_mask=self.gen_mask_for_src(src)
        # using in-built transformer function to generate mask for english (target)
        # It will be in form of a lower-triangular matrix
        tgt_mask=self.transformer_layer.generate_square_subsequent_mask(tgt_seq_length).to(self.device)
        output=self.transformer_layer(src_embedding, tht_embedding, src_key_padding_mask=src_padding_mask, tgt_mask=tgt_mask)
        output=self.out_fc(output)
        return output

**Training the Model**

In [ ]:
# defining model parameters
embed_size=512
len_hin_vocab=hindi_vocab.size
len_eng_vocab=eng_vocab.size
padding_idx=eng_vocab.word_2_index["<PAD>"]
num_heads=8
enc_layers, dec_layers= 1,1
dropout=0.10
forward_expansion=4

In [ ]:
model=Transformer_model(embed_size, len_eng_vocab, len_hin_vocab, padding_idx, num_heads, enc_layers, dec_layers, forward_expansion, dropout, MAX_LENGTH+2,device).to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
model_available=False # A variable to indicate whether a model is present in the path or not

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
import time
batch_size=128
optimizer=optim.Adam(model.parameters(),lr=0.001)
PATH="model.pth"

epochs=20
epoch_loss=0.0

criterion=nn.CrossEntropyLoss(ignore_index=padding_idx) #ignore padding index while calculating loss


if model_available:
    model=torch.load(PATH)
batches=len(pairs)//batch_size


start_time = time.time()


for epoch in range(epochs):
    epoch_start = time.time()
    print(f"epoch {epoch+1}/{epochs}")
    model.eval()
    model.train(True)
    cur_batch=0
    batch_start = time.time()
    for idx in range(0,len(pairs),batch_size):
        # will do processing for each batch
        cur_batch+=1
        if(cur_batch%100==0):
            print(f"    running batch {cur_batch} of {batches}")
        if idx+batch_size < len(pairs):
            src_batch=eng_tensors[idx:idx+batch_size]
            target_batch=hin_tensors[idx:idx+batch_size]
        else:
            src_batch=eng_tensors[idx:]
            target_batch=hin_tensors[idx:]

        src_batch=torch.cat(src_batch,dim=1).to(device)    #shape: (max_lenbatch_size)
        target_batch=torch.cat(target_batch,dim=1).to(device) #shape: (max_lenbatch_size)
        output=model(src_batch,target_batch[:-1,:])
        output=output.reshape(-1, output.shape[2])

        target=target_batch[1:].reshape(-1)

        optimizer.zero_grad()
        loss=criterion(output,target)

        loss.backward()
        # restrict gradients from exploding
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()
        epoch_loss += loss.item()

    print('%s (%d %d%%) %.4f' % (timeSince(start_time, (epoch+1) / epochs),
                                        epoch+1, (epoch+1) / epochs * 100, loss.item()))

    torch.save(model,PATH)
    model_available=True

epoch 1/20
    running batch 100 of 1061
    running batch 200 of 1061
    running batch 300 of 1061
    running batch 400 of 1061
    running batch 500 of 1061
    running batch 600 of 1061
    running batch 700 of 1061
    running batch 800 of 1061
    running batch 900 of 1061
    running batch 1000 of 1061
4m 45s (- 90m 15s) (1 5%) 6.5670
epoch 2/20
    running batch 100 of 1061
    running batch 200 of 1061
    running batch 300 of 1061
    running batch 400 of 1061
    running batch 500 of 1061
    running batch 600 of 1061
    running batch 700 of 1061
    running batch 800 of 1061
    running batch 900 of 1061
    running batch 1000 of 1061
9m 37s (- 86m 35s) (2 10%) 5.6628
epoch 3/20
    running batch 100 of 1061
    running batch 200 of 1061
    running batch 300 of 1061
    running batch 400 of 1061
    running batch 500 of 1061
    running batch 600 of 1061
    running batch 700 of 1061
    running batch 800 of 1061
    running batch 900 of 1061
    running batch 1000 of 10

In [37]:
def clean_sentence(sentence):
    '''
      Function to remove the punctuation from the test sentence
    '''
    punctuations=list(string.punctuation)
    cleaned=""
    for letter in sentence:
        if letter=='<' or letter=='>' or letter not in punctuations:
            cleaned+=letter
    return cleaned

def predict_translation(model,sentence,device,max_length=MAX_LENGTH):
    '''
      function will return the translation predicted by the trained model for each sentence
    '''
    sentence=clean_sentence(sentence)
    tokens=sentence.split(" ")
    indexes=[]
    for token in tokens:
        if token in eng_vocab.word_2_index:
            indexes.append(eng_vocab.word_2_index[token])
        else:
            indexes.append(eng_vocab.word_2_index["<UKN>"])
    indexes=indexes[:MAX_LENGTH+2]  # model is trained on MAX_LENGTH sentences only so it expects sentences of this length only
    tensor_of_sentence=torch.LongTensor(indexes).unsqueeze(1).to(device)
    outputs=[0]   #adding <SOS> in the beginning of output
    for _ in range(max_length):
        target_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)
        with torch.no_grad():
            output=model(tensor_of_sentence,target_tensor)
        pred=output.argmax(2)[-1, :].item()

        outputs.append(pred)

        if hindi_vocab.index_2_word[pred] =="<EOS>":
            break

    final=[]

    for i in outputs:
        if i == "<PAD>":
            break
        final.append(i)

    final = [hindi_vocab.index_2_word[idx] for idx in final if idx not in [0,1,2]]
    translated=" ".join(final)
    return translated

In [51]:
def generateEnglishSentence(sentence):
    extra_tokens = MAX_LENGTH-len(sentence.strip().split(" "))
    cleanedSentence = sentence.split(" ")
    cleanedSentence.insert(0,"<SOS>")
    cleanedSentence.append("<EOS>")
    cleanedSentence = cleanedSentence +["<PAD>"]*(extra_tokens)
    return " ".join(cleanedSentence)


def generate_translation_from_english(model , sentence , device):
  sentence = generateEnglishSentence(sentence)
  return predict_translation(model , sentence , device)

generate_translation_from_english(model , "how are you" , device)

'आप किस तरह से'

In [42]:
# Checking how the model is translating sentences from the train set
test_sentences=[pair[1] for pair in pairs[75:150]]
actual_sentences=[pair[0] for pair in pairs[75:150]]
pred_sentences=[]

for idx,i in enumerate(test_sentences):
    translated=predict_translation(model,i,device)
    print("*"*20)
    print(f"English: {i}")
    print(f"Actual: {actual_sentences[idx]}")
    print(f"Predicted: {translated}")
    print("*"*20)

********************
English: <SOS> survival skills <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Actual: <SOS> उत्तरजीविता कौशल <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Predicted: उत्तरजीविता कौशल
********************
********************
English: <SOS> sociology of culture <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Actual: <SOS> संस्कृति का समाजशास्‍त्र <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Predicted: संस्कृति का समाजशास्‍त्र
********************
********************
English: <SOS> load the last project that was not burned and not saved <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Actual: <SOS> अंतिम परियोजना लोड करें जो नहीं लिखी और नहीं सहेजी गई थी <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 